# Debugging C

In [ ]:
import os
# The jupyter notebook is launched from your $HOME directory.
# Change the working directory to the C-Debugging directory
# which was created in your username directory under /scratch/vp91
os.chdir(os.path.expandvars("/scratch/vp91/$USER/C-Debugging/"))

## 1. gdb
To use `gdb`, first we need to compile with debug symbols (`-g` flag)

In [ ]:
!gcc test_gdb.c list.c -o test_gdb -g -Wall -Wextra -Wpedantic

Now run under `gdb` with `gdb ./test_gdb`. This doesn't work nicely in a Jupyter notebook, so easiest to use a Gadi terminal, `cd` into `/scratch/vp91/$USER/C-Debugging`, and run the command there.

It can be useful to have the source code ([test_gdb.c](./test_gdb.c) and [list.c](./list.c)) open at the same time to help keep track of where you're up to as the program runs.

Check [here](http://users.ece.utexas.edu/~adnan/gdb-refcard.pdf) for a handy `gdb` command reference.

### 1.1 list_pop()
Next, let's add the `list_pop` method to [list.c](./list.c) and [list.h](./list.h) and check using `gdb` whether it's working correctly.

It should have the signature:
```C
double list_pop(List* list, const size_t index);
```
That is, it takes a list and an index, removes the item at `index` from the list, and returns that item.

In [ ]:
!gcc test_list_pop.c list.c -o test_list_pop -g -Wall -Wextra -Wpedantic

## 2. valgrind
`valgrind` also makes use of the `-g` flag to give more useful diagnostic messages.

Firstly, try changing [test_list_pop.c](./test_list_pop.c) to pop element 10. This is 1 element beyond the length of the list, but let's see what hapens.

After making the change, recompile with:

In [ ]:
!gcc test_list_pop.c list2.c -o test_list_pop2 -g -Wall -Wextra -Wpedantic

Running normally, it looks like it just popped the last element, like we may be expecting if we didn't realise that 10 was one beyond the length of the array.

In [ ]:
!./test_list_pop2

Now try running with `valgrind` (or equivalently, `valgrind --tool=memcheck`)

In [ ]:
!valgrind --tool=memcheck ./test_list_pop2

We have an invalid read! When we read `list->data[index]`, `index` is larger than the allocated space for `list->data`, so we're accessing invalid memory which is certainly a bug!

`valgrind` is also good at detecting memory leaks. Try commenting out the `list_destroy(&squares)` line and see what happens when you compile and re-run:

In [ ]:
!gcc test_list_pop.c list2.c -o test_list_pop3 -g -Wall -Wextra -Wpedantic

In [ ]:
!./test_list_pop3

In [ ]:
!valgrind ./test_list_pop3

Notice the `LEAK SUMMARY` section. It says we've definitely lost 72 bytes, which is exactly the size of the list after one element has been removed (8 bytes per item * 9 items). This is because without the call to `list_destroy`, the heap memory used by the list is never freed.

While running under `valgrind` is slower than normal execution, it can be a useful way of detecting potential bugs, including ones that may be inconsequential now, but end up causing problems later as the code evolves.

### 2.1 Debugging a Finite Difference Code
[finite_difference.c](./finite_difference.c) contains some code for calculating the derivative of a function over a given range and saving it to a .csv file, but it contains some bugs!

Using `gdb` and `valgrind`, see if you can get it working correctly.
You can check a plot of the result with the python block below.

Also note in the compile command below that we're using the `-lm` flag to link the math library (accessed via `math.h`).

In [ ]:
!gcc finite_difference.c -o finite_difference -g -lm -Wall -Wextra -Wpedantic

In [ ]:
!./finite_difference fd_output.csv

In [ ]:
from numpy import genfromtxt
import matplotlib.pyplot as plt
data = genfromtxt('fd_output.csv', delimiter=',')
plt.plot(data[:,0], data[:,1], label='y(x)')
plt.plot(data[:,0], data[:,2], label="y'(x)")
plt.legend()
plt.xlabel('x')
plt.ylabel('y')

#### HINT:

Finite difference stencils are:
* Forward: f'(x) = (f(x+h) - f(x)) / h
* Backward: f'(x) = (f(x) - f(x-h)) / h
* Central: f'(x) = (f(x+h) - f(x-h)) / 2h

## 3. Arm Forge
Edit [test_list_insert.c](./test_list_insert.c) to add a `list_insert` function, which inserts a new element into the list. For example:
```C
// list contains [1.0, 2.0, 3.0]
list_insert(&list, 1, 1.5);
// list contains [1.0, 1.5, 2.0, 3.0]
```
The function signature should be:
```C
void list_insert(List* list, size_t index, double value);
```

Try debugging with Arm Forge to fix any issues and verify that it's working correctly without any memory bugs.

See the workshop slides for instructions on how to access Arm Forge.

In [ ]:
!gcc test_list_insert.c list2.c -o test_list_insert -g -Wall -Wextra -Wpedantic

In [ ]:
!./test_list_insert

### 3.1 Multithreading
[test_threaded.c](./test_threaded.c) contains a simple multithreaded summing routine.
Try compiling it with the command below and running it within Arm Forge to test out its tools for debugging with multiple threads.

In [ ]:
!gcc test_threaded.c -o test_threaded -g -Wall -Wextra -Wpedantic -lpthread

In [ ]:
!./test_threaded